En este notebook utilizamos la API de Spotify para conseguir la variable 'valencia', la cual mide cómo de positiva es una canción según sus características musicales. Como cualquier API, requiere claves de acceso. Como en otros casos, el código está interrumpido tras comprobar que sigue funcionando correctamente.

### Librerías

In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import os
import requests
import time
import random 

In [2]:
names = lambda o: print([name for name in dir(o) if name[0] != "_"])
names([])

['append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


In [3]:
#### Usuario Spotify developer
CLIENT_ID = 'XXXXXXX'
CLIENT_SECRET = 'XXXXXXX'

### Prueba con 1 canción

In [4]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [5]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [6]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# Track ID from the URI
track_id = '6y0igZArWVi6Iz0rj35c1Y'

# actual GET request with proper header
r = requests.get(BASE_URL + 'audio-features/' + track_id, headers=headers)

In [7]:
r = r.json()
r

{'danceability': 0.54,
 'energy': 0.59,
 'key': 0,
 'loudness': -4.359,
 'mode': 1,
 'speechiness': 0.0528,
 'acousticness': 0.446,
 'instrumentalness': 0,
 'liveness': 0.14,
 'valence': 0.267,
 'tempo': 119.878,
 'type': 'audio_features',
 'id': '6y0igZArWVi6Iz0rj35c1Y',
 'uri': 'spotify:track:6y0igZArWVi6Iz0rj35c1Y',
 'track_href': 'https://api.spotify.com/v1/tracks/6y0igZArWVi6Iz0rj35c1Y',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6y0igZArWVi6Iz0rj35c1Y',
 'duration_ms': 234910,
 'time_signature': 4}

In [8]:
df2 = json_normalize(r) 
df2

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.54,0.59,0,-4.359,1,0.0528,0.446,0,0.14,0.267,119.878,audio_features,6y0igZArWVi6Iz0rj35c1Y,spotify:track:6y0igZArWVi6Iz0rj35c1Y,https://api.spotify.com/v1/tracks/6y0igZArWVi6...,https://api.spotify.com/v1/audio-analysis/6y0i...,234910,4


### Spotify API para todas las canciones

In [3]:
df1 = pd.read_excel('./data/total_songs.xlsx')
df1

,Song,Artist,URL,SongID
0,Fake Love,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,Fake LoveDrake
1,Bad and Boujee (feat. Lil Uzi Vert),Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,Bad and Boujee (feat. Lil Uzi Vert)Migos
2,Starboy,The Weeknd,https://open.spotify.com/track/5aAx2yezTd8zXrk...,StarboyThe Weeknd
3,Black Beatles,Rae Sremmurd,https://open.spotify.com/track/6fujklziTHa8uoM...,Black BeatlesRae Sremmurd
4,Closer,The Chainsmokers,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,CloserThe Chainsmokers
...,...,...,...,...
49539,Yalan,Tan Taşçı,https://open.spotify.com/track/34068op0PV7OoOf...,YalanTan Taşçı
49540,Yalan - Akustik,Tan Taşçı,https://open.spotify.com/track/5ECYDjSVrW99wt1...,Yalan - AkustikTan Taşçı
49541,KISKANMA,Sdrk,https://open.spotify.com/track/3aBWFcUEAXHaBFb...,KISKANMASdrk
49542,Oyunbozan,mor ve ötesi,https://open.spotify.com/track/4Po5JZuDOXRGHFt...,Oyunbozanmor ve ötesi


In [4]:
# Lista de las variables a añadir al dataframe
variables = ['valence', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'tempo', 'duration_ms', 'time_signature']

# Añadir las columnas con valores NaN a df1
df1[variables] = pd.DataFrame([[np.nan] * len(variables)], index=df1.index)

df1

,Song,Artist,URL,SongID,valence,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,duration_ms,time_signature
0,Fake Love,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,Fake LoveDrake,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Bad and Boujee (feat. Lil Uzi Vert),Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,Bad and Boujee (feat. Lil Uzi Vert)Migos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Starboy,The Weeknd,https://open.spotify.com/track/5aAx2yezTd8zXrk...,StarboyThe Weeknd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Black Beatles,Rae Sremmurd,https://open.spotify.com/track/6fujklziTHa8uoM...,Black BeatlesRae Sremmurd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Closer,The Chainsmokers,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,CloserThe Chainsmokers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49539,Yalan,Tan Taşçı,https://open.spotify.com/track/34068op0PV7OoOf...,YalanTan Taşçı,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49540,Yalan - Akustik,Tan Taşçı,https://open.spotify.com/track/5ECYDjSVrW99wt1...,Yalan - AkustikTan Taşçı,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49541,KISKANMA,Sdrk,https://open.spotify.com/track/3aBWFcUEAXHaBFb...,KISKANMASdrk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49542,Oyunbozan,mor ve ötesi,https://open.spotify.com/track/4Po5JZuDOXRGHFt...,Oyunbozanmor ve ötesi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Define las credenciales
CLIENT_ID = 'XXXXXX'
CLIENT_SECRET = 'XXXXXX'
AUTH_URL = 'https://accounts.spotify.com/api/token'

# Obtiene el access token
def get_access_token(client_id, client_secret):
    """Obtiene un access token de Spotify"""
    auth_url = 'https://accounts.spotify.com/api/token'
    response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    if response.status_code != 200:
        raise Exception(f"No se pudo obtener el token de acceso. status={response.status_code}, message={response.text}")
    return response.json()['access_token']


# Función para obtener los audio features con reintento
def get_audio_features_retry(track_id, access_token, retries=3):
    """Obtiene los audio features de una canción de Spotify, con reintento en caso de error"""
    while retries > 0:
        try:
            url = f"https://api.spotify.com/v1/audio-features/{track_id}"
            headers = {'Authorization': f'Bearer {access_token}'}
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response.json()
            elif response.status_code == 401:  # Token expirado
                access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)
                return get_audio_features_retry(track_id, access_token, retries-1)  # Reintenta la petición con el nuevo token
            else:
                retries -= 1
                time.sleep(random.randrange(5, 10)) # Espera un tiempo aleatorio antes de reintentar
        except:
            retries -= 1
            time.sleep(random.randrange(5, 10)) # Espera un tiempo aleatorio antes de reintentar
    return None


# Obtiene el access token inicial
access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

# Bucle while para buscar características de audio faltantes
while df1[['valence', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
           'instrumentalness', 'liveness', 'tempo', 'duration_ms', 'time_signature']].isna().sum().sum() > 0:
    for i in range(len(df1)):
        row = df1.iloc[i]
        if pd.isna(row['valence']) or pd.isna(row['danceability']) or pd.isna(row['energy']) or pd.isna(row['key']) or pd.isna(row['loudness']) or pd.isna(row['mode']) or pd.isna(row['speechiness']) or pd.isna(row['acousticness']) or pd.isna(row['instrumentalness']) or pd.isna(row['liveness']) or pd.isna(row['tempo']) or pd.isna(row['duration_ms']) or pd.isna(row['time_signature']):
            print(i)
            time.sleep(random.randrange(0, 2))
            track_id = row['URL'][31:]
            audio_features = get_audio_features_retry(track_id, access_token)
            if audio_features is not None:
                valid_cols = set(df1.columns).intersection(audio_features.keys())
                df1.loc[i, list(valid_cols)] = [audio_features[k] for k in valid_cols]
            else:
                time.sleep(random.randrange(70, 100))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [13]:
df1.head(50)

,Song,Artist,URL,SongID,valence,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,duration_ms,time_signature
0,Fake Love,Drake,https://open.spotify.com/track/343YBumqHu19cGo...,Fake LoveDrake,0.6130,0.928,0.481,9.0,-9.350,0.0,0.2870,0.10500,0.000000,0.1760,134.007,210937.0,4.0
1,Bad and Boujee (feat. Lil Uzi Vert),Migos,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,Bad and Boujee (feat. Lil Uzi Vert)Migos,0.1680,0.926,0.666,11.0,-5.314,1.0,0.2440,0.06110,0.000000,0.1230,127.079,343150.0,4.0
2,Starboy,The Weeknd,https://open.spotify.com/track/5aAx2yezTd8zXrk...,StarboyThe Weeknd,0.5350,0.681,0.594,7.0,-7.028,1.0,0.2820,0.16500,0.000003,0.1340,186.054,230453.0,4.0
3,Black Beatles,Rae Sremmurd,https://open.spotify.com/track/6fujklziTHa8uoM...,Black BeatlesRae Sremmurd,0.3550,0.794,0.632,0.0,-6.163,1.0,0.0649,0.14200,0.000000,0.1280,145.926,291893.0,4.0
4,Closer,The Chainsmokers,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,CloserThe Chainsmokers,0.6610,0.748,0.524,8.0,-5.599,1.0,0.0338,0.41400,0.000000,0.1110,95.010,244960.0,4.0
5,Caroline,Aminé,https://open.spotify.com/track/5hTpBe8h35rJ67e...,CarolineAminé,0.7070,0.940,0.335,5.0,-10.179,0.0,0.5050,0.17000,0.000000,0.2620,120.040,209640.0,4.0
6,Bad Things (with Camila Cabello),Machine Gun Kelly,https://open.spotify.com/track/1PSBzsahR2AKwLJ...,Bad Things (with Camila Cabello)Machine Gun Kelly,0.3050,0.697,0.691,2.0,-4.757,1.0,0.1460,0.21400,0.000000,0.1850,137.853,239293.0,4.0
7,I Don’t Wanna Live Forever (Fifty Shades Darke...,ZAYN,https://open.spotify.com/track/3NdDpSvN911VPGi...,I Don’t Wanna Live Forever (Fifty Shades Darke...,0.0862,0.735,0.451,0.0,-8.374,1.0,0.0585,0.06310,0.000013,0.3250,117.973,245200.0,4.0
8,Deja Vu,J. Cole,https://open.spotify.com/track/6I6NX6tjGsxFAsI...,Deja VuJ. Cole,0.1810,0.657,0.421,9.0,-11.378,1.0,0.4120,0.35200,0.000218,0.1870,174.063,264747.0,4.0
9,Broccoli (feat. Lil Yachty),Shelley FKA DRAM,https://open.spotify.com/track/7yyRTcZmCiyzzJl...,Broccoli (feat. Lil Yachty)Shelley FKA DRAM,0.7080,0.886,0.525,8.0,-7.390,1.0,0.1310,0.23600,0.000000,0.0570,145.990,225205.0,4.0


In [16]:
df1.to_excel('valence.xlsx')